# Leitura dos arquivos em Verilog, Construção da rede bayseana e Analise dela através da inferencia

In [1]:
# Importação das bibliotecas
import pyAgrum as gum # pyagrum 
import pyAgrum.lib.notebook as gnb
import matplotlib.pyplot as plt
from pylab import *
from IPython.display import HTML, display
import itertools
import sys 
gum.config['notebook','potential_visible_digits']=2


In [2]:
# Escrever no arquivo (função traduzida do código do kenner)

def write_to_file(vec_of_vecs, arquivo):
    """
    Escreve uma lista de listas de strings em um arquivo separando os elementos por espaços
    e cada lista interna por uma nova linha

    Parâmetros:
    vec_of_vecs (List[List[str]]): A lista de listas de strings a ser escrita no arquivo
    arquivo (str): O caminho para o arquivo onde os dados serão escritos
    """
    try:
        with open(arquivo, 'w') as out_file:
            for inner_list in vec_of_vecs:
                # Escrever os elementos separados por espaço
                line = ' '.join(inner_list)
                out_file.write(line + '\n')
    except IOError as e:
        print(f"Erro ao abrir ou escrever no arquivo: {arquivo}\n{e}", file=sys.stderr)



In [3]:
# Função de ler o arquivo em verilog e retornar os elementos em uma estrutura de dados

def read_verilog(caminho_verilog):
    """
    Lê um arquivo Verilog e extrai argumentos lógicos de cada linha que contenha
    um parêntese aberto dentro das primeiras 5 posições

    Parametros:
    caminho_verilog (str): O caminho para o arquivo Verilog a ser lido

    Retorna:
    List[List[str]]: Uma lista de listas contendo os argumentos lógicos extraídos
    """
    data_vet = []
    
    try:
        with open(caminho_verilog, 'r') as arquivo_verilog:
            for line in arquivo_verilog:
                line = line.strip()
                if not line:
                    continue  # Pula linhas vazias

                # Encontrar a posição do primeiro parêntese
                parenthesis_pos = line.find('(')
                if parenthesis_pos != -1 and parenthesis_pos < 5:
                    # Extrair o argumento lógico antes do parêntese
                    logic_argument = line[:parenthesis_pos].strip()
                    temp_vector = [logic_argument]

                    # Extrair o conteúdo dentro dos parênteses
                    inside_parentheses = line[parenthesis_pos+1:]
                    # Remover tudo após o último parêntese
                    end_parenthesis_pos = inside_parentheses.find(')')
                    if end_parenthesis_pos != -1:
                        arguments_str = inside_parentheses[:end_parenthesis_pos]
                        # Dividir os argumentos por vírgula
                        arguments = [arg.strip() for arg in arguments_str.split(',')]
                        temp_vector.extend(arguments)
                    
                    data_vet.append(temp_vector)
    except IOError as e:
        print(f"Erro ao abrir o arquivo: {caminho_verilog}\n{e}", file=sys.stderr)
        return []

    return data_vet


In [4]:
# Construção do código de ler o txt do verilog e criar a rede bayseana
# 1° Parte: estudo de como ira retornar o txt 
with open('verilog/C1355.txt', 'r') as f:

    for linha in f: 
        linha = linha.strip()
        if not linha:
            continue 

        partes = linha.split()
        operacao = partes[0] # porta lógica
        saida = partes[1] # output
        entradas = partes[2:] # input
        print(f'\nPartes: {partes}\nOperações: {partes[0]}\nSaida: {partes[1]}')
        if operacao == 'or' or operacao == 'and':
            print(f'\nEntradas 1: {entradas[0]}\nEntradas 2: {entradas[1]}')
        else:
            print(f'\nEntrada: {entradas[0]}')




Partes: ['and', 'v1', 'n0', 'o0']
Operações: and
Saida: v1

Entradas 1: n0
Entradas 2: o0

Partes: ['and', 'w1', 'm0', 'o0']
Operações: and
Saida: w1

Entradas 1: m0
Entradas 2: o0

Partes: ['and', 'x1', 'l0', 'o0']
Operações: and
Saida: x1

Entradas 1: l0
Entradas 2: o0

Partes: ['and', 'y1', 'k0', 'o0']
Operações: and
Saida: y1

Entradas 1: k0
Entradas 2: o0

Partes: ['and', 'z1', 'j0', 'o0']
Operações: and
Saida: z1

Entradas 1: j0
Entradas 2: o0

Partes: ['and', 'a2', 'i0', 'o0']
Operações: and
Saida: a2

Entradas 1: i0
Entradas 2: o0

Partes: ['and', 'b2', 'h0', 'o0']
Operações: and
Saida: b2

Entradas 1: h0
Entradas 2: o0

Partes: ['and', 'c2', 'g0', 'o0']
Operações: and
Saida: c2

Entradas 1: g0
Entradas 2: o0

Partes: ['not', 't_0', 'f0']
Operações: not
Saida: t_0

Entrada: f0

Partes: ['not', 't_1', 'e0']
Operações: not
Saida: t_1

Entrada: e0

Partes: ['or', 'd2', 't_0', 't_1']
Operações: or
Saida: d2

Entradas 1: t_0
Entradas 2: t_1

Partes: ['not', 't_2', 'd0']
Operações: 

In [101]:
# 2° Parte: adicionando a porta lógica a BN

def add_port_logic (bn, operacao, output, inputs):
    """
    Adiciona a porta lógica a rede bayseana 

    parametros->
    bn: rede bayseana
    operacao: porta lógica (or, not, and)
    output: variavel de saida
    input: variaveis de entrada (lista)
    """

    operacao = operacao.lower() # evitando erro por maiusculo

    # criação das variaveis de entrada

    for inp in inputs: 
        if inp not in bn.names(): # evita de criar variaveis já criadas na rede bayseana
            variavel = gum.LabelizedVariable(inp, inp, 2)
            # variavel = bn.add(gum.LabelizedVariable(inp,inp,2))
            bn.add(variavel)
            # criação da cpt
            bn.cpt(inp).fillWith([0.5,0.5])


        # criar variavel da saida
    if output not in bn.names():
        variavel_output = gum.LabelizedVariable(output, output, 2)
        bn.add(variavel_output)


    # add arcos da entrada para a saida
    for inp in inputs:
        bn.addArc(inp, output)

    # Definir a CPT com base na operação lógica
    if operacao == "not":
        if len(inputs) != 1:
            raise ValueError("A operação 'not' requer exatamente 1 entrada.")

        inp = inputs[0]
        # Porta NOT: Y = NOT X
        # Definindo P(Y=1 | X=0) = 1.0 e P(Y=0 | X=0) = 0.0
        # Definindo P(Y=1 | X=1) = 0.0 e P(Y=0 | X=1) = 1.0
        bn.cpt(output)[{inp: 0}] = [0.0, 1.0]  # Y=0, Y=1
        bn.cpt(output)[{inp: 1}] = [1.0, 0.0]  # Y=0, Y=1
        
    elif operacao == "or":
        if len(inputs) != 2:
            raise ValueError("Erro brabo")
        # Porta OR: Y = X1 OR X2 OR ...
        #                                              [0, 1] -> bits
        bn.cpt(output)[{inputs[0]: 0, inputs[1]: 0}] = [1.0, 0.0]  # P(Y=0|r=0,s=0) = 1.0, P(Y=1|r=0,s=0) = 0.0
        bn.cpt(output)[{inputs[0]: 0, inputs[1]: 1}] = [0.0, 1.0]  # P(Y=0|r=0,s=1) = 0.0, P(Y=1|r=0,s=1) = 1.0
        bn.cpt(output)[{inputs[0]: 1, inputs[1]: 0}] = [0.0, 1.0]  # P(Y=0|r=1,s=0) = 0.0, P(Y=1|r=1,s=0) = 1.0
        bn.cpt(output)[{inputs[0]: 1, inputs[1]: 1}] = [0.0, 1.0]  # P(Y=0|r=1,s=1) = 0.0, P(Y=1|r=1,s=1) = 1.0

    elif operacao == "and":
        # Porta AND: Y = X1 AND X2 AND ...
        bn.cpt(output)[{inputs[0]: 0, inputs[1]: 0}] = [1.0, 0.0]  # P(Y=0|r=0,s=0) = 1.0, P(Y=1|r=0,s=0) = 0.0
        bn.cpt(output)[{inputs[0]: 0, inputs[1]: 1}] = [1.0, 0.0]  # P(Y=0|r=0,s=1) = 0.0, P(Y=1|r=0,s=1) = 0.0
        bn.cpt(output)[{inputs[0]: 1, inputs[1]: 0}] = [1.0, 0.0]  # P(Y=0|r=1,s=0) = 0.0, P(Y=1|r=1,s=0) = 0.0
        bn.cpt(output)[{inputs[0]: 1, inputs[1]: 1}] = [0.0, 1.0]  # P(Y=0|r=1,s=1) = 0.0, P(Y=1|r=1,s=1) = 1.0

    else:
        raise ValueError(f"Operação lógica '{operacao}' não suportada.")
    


In [118]:
# teste com circuito pequeno de uma porta
bn = gum.BayesNet('test')
input = ['s0', 's1']
output = 'r0'
operacao = 'and'
add_port_logic(bn,operacao=operacao, output=output, inputs=input)

gnb.sideBySide(bn.cpt(output), gnb.getBN(bn, size='1.9!'), captions=[f'Porta {operacao}','Rede Bayseana'])

In [89]:
# 3° Parte: criando com base no txt

def build_bn(path_txt):
    """
    Constrói uma rede bayesiana a partir de definições de portas lógicas em texto.

    Parâmetros:
    - gate_definitions_text: String contendo as definições das portas lógicas, uma por linha.

    Retorna:
    - bn: Instância da rede bayesiana construída (gum.BayesNet).
    """
    bn = gum.BayesNet('Circuito_Combinacional')

    # Processar cada linha de definição de porta lógica
    with open(path_txt, 'r') as f:

        for linha in f: 
            linha = linha.strip()
            if not linha:
                continue 

            partes = linha.split()
            operacao = partes[0] # porta lógica
            saida = partes[1] # output
            entradas = partes[2:] # input
            add_port_logic(bn, operacao=operacao, output=saida, inputs=entradas)

    return bn

In [108]:
# teste com meio somador
write_to_file(read_verilog('verilog/somador2bits.v'), 'data/somador2bits.txt')
bn2 = build_bn('data/somador2bits.txt')
gum.saveBN(bn,"data/somador2bits.xdsl")

bn2



(pyAgrum.BayesNet<double>@0x316e8e0) BN{nodes: 13, arcs: 14, domainSize: 8192, dim: 32, mem: 512o}

In [102]:
bn3 = build_bn('verilog/C1355.txt')
gum.saveBN(bn3, 'data/C1355.xdsl')

IndexError: Shape of '[1. 0.]' is not '(2, 2, 2)'